In [1]:
import torch
from torchvision import datasets, transforms

# Veri dönüşümleri
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Fashion-MNIST veri seti için normalize işlemi
])

# Fashion-MNIST veri setini yükleme
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Veri yükleyicilerini oluşturma
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


100%|██████████| 26421880/26421880 [00:02<00:00, 11765553.65it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 206719.64it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3889760.05it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 3988966.75it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


# Sinir ağı modeli
class ClassifierModule(nn.Module):
    def __init__(self):
        super(ClassifierModule, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.softmax(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ClassifierModule().to(device)

# Loss fonksiyonu ve optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Model eğitimi
num_epochs = 100

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 1/100: Train Loss: 1.6857, Test Loss: 1.6688, Test Accuracy: 0.7900
Epoch 2/100: Train Loss: 1.6415, Test Loss: 1.6509, Test Accuracy: 0.8100
Epoch 3/100: Train Loss: 1.6352, Test Loss: 1.6329, Test Accuracy: 0.8280
Epoch 4/100: Train Loss: 1.6291, Test Loss: 1.6341, Test Accuracy: 0.8261
Epoch 5/100: Train Loss: 1.6287, Test Loss: 1.6521, Test Accuracy: 0.8090
Epoch 6/100: Train Loss: 1.6247, Test Loss: 1.6372, Test Accuracy: 0.8232
Epoch 7/100: Train Loss: 1.6265, Test Loss: 1.6364, Test Accuracy: 0.8241
Epoch 8/100: Train Loss: 1.6227, Test Loss: 1.6362, Test Accuracy: 0.8245
Epoch 9/100: Train Loss: 1.6214, Test Loss: 1.6202, Test Accuracy: 0.8407
Epoch 10/100: Train Loss: 1.6234, Test Loss: 1.6311, Test Accuracy: 0.8297
Epoch 11/100: Train Loss: 1.6219, Test Loss: 1.6234, Test Accuracy: 0.8367
Epoch 12/100: Train Loss: 1.6201, Test Loss: 1.6291, Test Accuracy: 0.8310
Epoch 13/100: Train Loss: 1.6153, Test Loss: 1.6334, Test Accuracy: 0.8273
Epoch 14/100: Train Loss: 1.6119, 

In [4]:
def apply_lorentz_transformation(data):
    # Lorentz dönüşümü uygulama kodu
    beta = 0.99
    gamma = 1 / (torch.sqrt(torch.tensor(1 - beta**2)) if beta < 1 else float('inf'))  # Lorentz faktörü

    data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür
    transformed_data = gamma * (data_tensor - beta * data_tensor)
    return transformed_data

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms


# Eğitim döngüsü
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(dataloader):
        data, target = data.to(device), target.to(device)

        # Lorentz dönüşümü uygulama
        data = apply_lorentz_transformation(data)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(dataloader)

# Evaluation fonksiyonu
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(dataloader):
            data, target = data.to(device), target.to(device)

            # Lorentz dönüşümü uygulama
            data = apply_lorentz_transformation(data)

            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item()
            _, predicted = output.max(1)
            correct += predicted.eq(target).sum().item()
            total += target.size(0)
    return running_loss / len(dataloader), correct / total

# Model eğitimi
num_epochs = 100

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Epoch {epoch+1}/{num_epochs}: Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

<ipython-input-4-ca63d6f88a5e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data, dtype=torch.float32)  # Data'yı Tensor'a dönüştür


Epoch 1/100: Train Loss: 1.5955, Test Loss: 1.6118, Test Accuracy: 0.8488
Epoch 2/100: Train Loss: 1.5904, Test Loss: 1.6060, Test Accuracy: 0.8544
Epoch 3/100: Train Loss: 1.5896, Test Loss: 1.6073, Test Accuracy: 0.8536
Epoch 4/100: Train Loss: 1.5897, Test Loss: 1.6069, Test Accuracy: 0.8534
Epoch 5/100: Train Loss: 1.5871, Test Loss: 1.6000, Test Accuracy: 0.8604
Epoch 6/100: Train Loss: 1.5854, Test Loss: 1.6010, Test Accuracy: 0.8597
Epoch 7/100: Train Loss: 1.5847, Test Loss: 1.6062, Test Accuracy: 0.8543
Epoch 8/100: Train Loss: 1.5832, Test Loss: 1.6031, Test Accuracy: 0.8576
Epoch 9/100: Train Loss: 1.5820, Test Loss: 1.6030, Test Accuracy: 0.8574
Epoch 10/100: Train Loss: 1.5795, Test Loss: 1.6038, Test Accuracy: 0.8565
Epoch 11/100: Train Loss: 1.5809, Test Loss: 1.5963, Test Accuracy: 0.8640
Epoch 12/100: Train Loss: 1.5767, Test Loss: 1.5977, Test Accuracy: 0.8624
Epoch 13/100: Train Loss: 1.5774, Test Loss: 1.6030, Test Accuracy: 0.8568
Epoch 14/100: Train Loss: 1.5777, 